<a href="https://colab.research.google.com/github/albertopierini/CVProject_measure_fetal_head/blob/master/df_mask_rcnn_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!git clone https://github.com/albertopierini/Mask_RCNN.git
#!pip install pycocotools
#!pip install mrcnn
!pip uninstall tensorflow
!pip install tensorflow-gpu==1.14


!pip install keras==2.2.0
print("Installed packages")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy
import scipy
import PIL
import tensorflow
import skimage
import h5py
import IPython
import cv2
import keras
import scipy
import matplotlib
import cython
import statistics

print("All dependencies are installed")

#Test ricerca GPU
device_name = tensorflow.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
import os
os.chdir('/content/drive/My Drive/mrcnn_testing')

In [ ]:
pwd

In [ ]:
import sys
import random
import math
import numpy as np
import pandas as pd
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import os
from pycocotools.coco import COCO
from config import Config

import warnings
warnings.filterwarnings("ignore")

ROOT_DIR = os.path.abspath("../")
sys.path.append(ROOT_DIR)

#import model as modellib
import utils
import model2 as modellib2
import visualize

DRIVE_DIR = '/content/drive/My Drive/CV Project/'

%matplotlib inline

# Directory to save logs and trained model
MODEL_DIR = os.path.join(DRIVE_DIR, "logs")

In [ ]:
#@title Prasad maria chiara

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat Jun 13 14:15:39 2020

@author: mariachiarafiorentino
"""


import numpy as np
import math
import cv2
from cmath import pi
from numpy import sqrt
from scipy.ndimage.interpolation import geometric_transform


def circumference(params,pixel):
    angle_deg = params[2]
    angle_corr = 0 -angle_deg + 90
    angle = np.radians(angle_corr)
    a = (params[1][0]/2)
    b = (params[1][1]/2)
    #print(pixel[i])
    #HC = (2*pi*sqrt(((a*a)+(b*b)))/(2))*float(pixel[i])
    HC = 2*pi*sqrt(((a*a)+(b*b))/(2*1.0))
    hc = HC*float(pixel)
    return hc

#PRASAD ELLIFIT
def backToOriginalSize(immage):
    shape0 = immage.shape[0]
    shape1 = immage.shape[1]
    desired_size_x = 540
    desired_size_y = 800
    ratio_x = float(desired_size_x)/shape0
    ratio_y = float(desired_size_y)/shape1
    new_size = tuple([int(shape0*ratio_x),int(shape1*ratio_y)])
    
    
    # new_size should be in (width, height) format
    
    im = cv2.resize(immage, (new_size[1], new_size[0]))
    
    delta_w = desired_size_x - new_size[0]
    delta_h = desired_size_y - new_size[1]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)
    
    color = [0, 0, 0]
    new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT,
        value=color)
    return new_im

def findMaxMinXY(rows,cols,contour):
    maxX = 0
    maxY = 0
    minY = rows
    #print(rows)
    minX = cols
    x = contour[:,0]
    y = contour[:,1]
    for i in range(0,len(contour)):
        x1 = x[i]
        y1 = y[i]
        if x1<minX:
            minX = x1
        if y1<minY:
            minY = y1
        if x1>maxX:
            maxX = x1
        if y1>maxY:
            maxY = y1
            
    minpt = (minX,minY)
    maxpt = (maxX,maxY)
    return (minpt,maxpt)

def generateMats(x,y):
    
    #x = contour[:,0,0]
    #y = contour[:,0,1]
    A = np.zeros((len(x),5))
    B = np.zeros((len(x),1))
    
    for i in range(0,len(x)):
        A[i,0] = x[i]*x[i]
        A[i,1] = 2.0*x[i]*y[i]
        A[i,2] = -2.0*x[i]
        A[i,3] = -2.0*y[i]
        A[i,4] = -1.0
        
        B[i,0] = -1.0*y[i]*y[i]
        
    return (A,B)

def ellipseMatrixOps(A,B):
    
    phi = np.array(np.dot((np.linalg.inv(np.dot(A.transpose(),A))),(np.dot(A.transpose(),B))))
    residue = np.linalg.norm((np.dot(A,phi) - B))/np.linalg.norm(B)
    return (A,B,phi,residue)

def addConstantToVector(meanValx, meanValy, contours):
    
    x = contours[:,0]
    x = x.astype(float)
    y = contours[:,1]
    y = y.astype(float)
    for i in range(0,len(contours)):
        x[i] -= meanValx
        y[i] -= meanValy
        
    return (x,y)

def computeEllipseParams(phi,meanValx, meanValy):
    
    #meanValx = 352
    #meanValy = 272
    a = phi[0,0]
    b = phi[1,0]
    c = phi[2,0]
    d = phi[3,0]
    e = phi[4,0]
    x = (c-(d*b))/(a-(b*b))
    y = ((a*d)-(c*b))/(a-(b*b))
    #x = 0
    #y = 0
    #x = 352
    #y = 272
    
    
    params = []
    temp1 = math.sqrt((np.dot((1-a),(1-a))+(4*(np.dot(b,b)))))
    temp2 = e + (y*y) + (x*x*a) + (2*b)
    temp3 = 1 + a
    
    Angle = -0.5*(math.atan2(2*b, 1-a))
    minorRadius = math.sqrt(abs((2*temp2)/(temp3+temp1)))
    majorRadius = math.sqrt(abs((2*temp2)/(temp3-temp1)))
    
    Centroidx = x
    Centroidy = y
    Centroidx += meanValx
    Centroidy += meanValy
    minorRadius = minorRadius*2
    majorRadius = majorRadius*2
    Angle = np.rad2deg(Angle)

    
    params = [(Centroidx, Centroidy), (majorRadius, minorRadius), Angle]
    #params1 = [(352, 272), (majorRadius, minorRadius), Angle]
    
    return (params)

def ellipseFit(rows, cols, contours):
    
    minpt,maxpt = findMaxMinXY(rows, cols, contours)
    minx = minpt[0]
    miny = minpt[1]
    maxx = maxpt[0]
    maxy = maxpt[1]
    meanValx = (minx+maxx)/2
    meanValy = (miny+maxy)/2
    x,y = addConstantToVector(meanValx,meanValy,contours)
    A,B = generateMats(x,y)
    A,B,phi,residue = ellipseMatrixOps(A,B)
    params = computeEllipseParams(phi,meanValx, meanValy)
    
    return params

In [ ]:
class FetalHeadConfiguration(Config):  
  NAME = "fetalHead"
  BACKBONE = "resnet101"
  GPU_COUNT = 1
  IMAGES_PER_GPU = 4
  NUM_CLASSES = 1 + 1 #Background + fetal head
  STEPS_PER_EPOCH = 369
  IMAGE_MIN_DIM = 512
  IMAGE_MAX_DIM = 512
  IMAGE_MIN_SCALE = 2.0
  MAX_GT_INSTANCES = 1
  DETECTION_MIN_CONFIDENCE = 0

config = FetalHeadConfiguration()
config.display()

In [ ]:
class InferenceConfig(FetalHeadConfiguration):
  GPU_COUNT = 1
  IMAGES_PER_GPU = 1

inference_config = InferenceConfig()
model = modellib2.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

model_path = '/content/drive/My Drive/fetalhead20200620T1301/mask_rcnn_fetalhead_0022.h5'
model.load_weights(model_path, by_name=True)

In [ ]:
#@title save_image function
from PIL import Image, ImageDraw, ImageFont

def save_image(image, image_name, boxes, masks, class_ids, scores, class_names, filter_classs_names=None,
              scores_thresh=0.1, save_dir=None, mode=0):
     
  mode_list = [0, 1, 2, 3]
  assert mode in mode_list, "mode's value should in mode_list %s" % str(mode_list)

  if save_dir is None:
    save_dir = os.path.join(os.getcwd(), "output")
    if not os.path.exists(save_dir):
      os.makedirs(save_dir)

  useful_mask_indices = []

  N = boxes.shape[0]
  if not N:
    print("\n*** No instances in image %s to draw *** \n" % (image_name))
    return
  else:
    assert boxes.shape[0] == masks.shape[-1] == class_ids.shape[0]

  for i in range(N):
    # filter
    class_id = class_ids[i]
    score = scores[i] if scores is not None else None
    if score is None or score < scores_thresh:
      continue

    label = class_names[class_id]
    if (filter_classs_names is not None) and (label not in filter_classs_names):
      continue

    if not np.any(boxes[i]):
      # Skip this instance. Has no bbox. Likely lost in image cropping.
      continue

    useful_mask_indices.append(i)

  if len(useful_mask_indices) == 0:
    print("\n*** No instances in image %s to draw *** \n" % (image_name))
    return

  colors = visualize.random_colors(len(useful_mask_indices))

  if mode != 3:
    masked_image = image.astype(np.uint8).copy()
  else:
    masked_image = np.zeros(image.shape).astype(np.uint8)

  if mode != 1:
    for index, value in enumerate(useful_mask_indices):
      masked_image = visualize.apply_mask(masked_image, masks[:, :, value], colors[index])

  masked_image = Image.fromarray(masked_image)

  if mode == 3:
    masked_image.save(os.path.join(save_dir, '%s.jpg' % (image_name)))
    return

  draw = ImageDraw.Draw(masked_image)
  colors = np.array(colors).astype(int) * 255

  for index, value in enumerate(useful_mask_indices):
    class_id = class_ids[value]
    score = scores[value]
    label = class_names[class_id]

    y1, x1, y2, x2 = boxes[value]
    if mode != 2:
      color = tuple(colors[index])
      draw.rectangle((x1, y1, x2, y2), outline=color)

      # Label
      #font = ImageFont.truetype('/Library/Fonts/Arial.ttf', 15)
      draw.text((x1, y1), "%s %f" % (label, score), (255, 255, 255))

  masked_image.save(os.path.join(save_dir, '%s.jpg' % (image_name)))

In [ ]:
import csv
csv_test_path = '/content/drive/My Drive/CV Project/csv/test_set_pixel_size.csv'
class_names = ['BG', 'fetal_head']
df_test = pd.read_csv(csv_test_path, sep = ',')

with open('/content/drive/My Drive/CV Project/csv/submission.csv', 'w') as csvfile:
  filewriter = csv.writer(csvfile)
  filewriter.writerow(['filename','center_x_mm','center_y_mm','semi_axes_a_mm','semi_axes_b_mm','angle_rad'])

for index, row in df_test.iterrows():
  #Caricamento US
  pixel_size = row['pixel size(mm)']
  us_test = skimage.io.imread('/content/drive/My Drive/CV Project/test_set/' + row['filename'])
  us_test_RGB = cv2.cvtColor(us_test, cv2.COLOR_GRAY2RGB)
  print(row['filename'])
  print(us_test.shape)

  #Test
  results = model.detect([us_test_RGB], verbose=1)
  r = results[0]
  found_mask = r['masks']

  if(r['scores'].shape[0] == 1):
    print("1 ROI")
    found_mask = r['masks'][:,:,0]
    found_roi = r['rois'][0]

  elif(r['scores'].shape[0] > 1):
    print("2 ROIs")
    found_mask = r['masks'][:,:,0]
    found_acc = r['scores'][0]
    found_roi = r['rois'][0]
    for j in range(r['scores'].shape[0]):
      if(r['scores'][j] > found_acc):
        found_acc = r['scores'][j]
        found_mask = r['masks'][:,:,j]
        found_roi = r['rois'][j]

  fig = plt.figure()
  #fig.suptitle('{}'.format('prova'))
  plt.imshow(us_test, 'gray', interpolation='none')
  plt.imshow(found_mask, 'jet', interpolation='none', alpha=0.3)

  plt.gca().set_axis_off()
  plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
              hspace = 0, wspace = 0)
  plt.margins(0,0)
  plt.gca().xaxis.set_major_locator(plt.NullLocator())
  plt.gca().yaxis.set_major_locator(plt.NullLocator())
  plt.savefig('/content/drive/My Drive/CV Project/image_result_heatmap/' + row['filename'], bbox_inches = 'tight', pad_inches = 0, dpi = 150)

  #Estrazione ellisse
  ret, binary_mask = cv2.threshold(found_mask,0.4,1,cv2.THRESH_BINARY)
  binary_mask = binary_mask.astype(np.uint8)

  ret, bw_mask = cv2.threshold(found_mask,0.4,255,cv2.THRESH_BINARY)
  cv2.imwrite('/content/drive/My Drive/CV Project/found_masks/' + row['filename'], bw_mask)

  #Erode
  kernel = np.ones((5,5),np.uint8)
  bw_mask_eroded = cv2.erode(bw_mask, kernel, iterations = 2)
  #cv2.imwrite('/content/drive/My Drive/CV Project/found_mask_eroded/' + row['filename'], bw_mask_eroded)
  ret, binary_mask_eroded = cv2.threshold(bw_mask_eroded,127,1,cv2.THRESH_BINARY)

  white_points = np.argwhere(binary_mask != 0)
  a = white_points[:,1]
  b = white_points[:,0]
  contours = np.column_stack((a,b))

  params = ellipseFit(540, 800, contours)

  # Save image ellipse
  d_centro_realex = params[0][0]
  d_centro_realey = params[0][1]
  d_x00 = int(round((d_centro_realex),2))
  d_y00 = int(round((d_centro_realey),2))
  d_semi_axes_a = int(round((params[1][0])/2,2))
  d_semi_axes_b = int(round((params[1][1])/2,2))

  cv2.ellipse(us_test_RGB, (d_x00,d_y00), (d_semi_axes_a,d_semi_axes_b), params[2], 0, 360, (255,0,0), thickness=5)
  cv2.imwrite('/content/drive/My Drive/CV Project/image_result_ellipse/'+row['filename'], us_test_RGB)

  # Nuova riga csv   
  centro_realex = params[0][0]
  centro_realey = params[0][1]
  pixel = float(pixel_size)
  x00 = str(round((centro_realex)*pixel,2))
  y00 = str(round((centro_realey)*pixel,2))
  semi_axes_a = str(round((params[1][0]*pixel)/2,2))
  semi_axes_b = str(round((params[1][1]*pixel)/2,2))
  angle_rad = str(round((params[2]*math.pi)/180,2))
    
  with open('/content/drive/My Drive/CV Project/csv/submission.csv', 'a', newline='') as csvfile:
    filewriter = csv.writer(csvfile)
    filewriter.writerow([row['filename'],x00,y00,semi_axes_a,semi_axes_b,angle_rad])
  
  bw_mask2 = cv2.cvtColor(bw_mask.astype(np.uint8), cv2.COLOR_GRAY2RGB)
  us_mask_ellipse = cv2.addWeighted(us_test_RGB,1.0,bw_mask2,0.2,0)
  cv2.ellipse(us_test_RGB, (d_x00,d_y00), (d_semi_axes_a,d_semi_axes_b), params[2], 0, 360, (255,0,0), thickness=5)
  cv2.imwrite('/content/drive/My Drive/CV Project/mask_ellipse_overlap/' + row['filename'], us_mask_ellipse)
  skimage.io.imshow(us_mask_ellipse)
